## QS Relevance Feedback Requirements
1. Relevance requires adherence to the entire query.
2. Long context with small relevant chunks are relevant.
3. Context that provides no answer can still be relevant.
4. Feedback mechanism should differentiate between seeming and actual relevance.
5. Relevant but inconclusive statements should get increasingly high scores as they are more helpful for answering the query.

In [1]:
import os
os.environ["OPENAI_API_KEY"] = "..."

In [2]:
# Imports main tools:
from trulens_eval.feedback import OpenAI
openai = OpenAI()
relevance = openai.qs_relevance

## Relevance requires adherence to the entire query.

In [3]:
def test_lowadherence_short():
    score = relevance("How many stomachs does a cow have?","Cows' diet relies primarily on grazing.")
    assert score >= 0.2, f"Score of {score} < 0.2. Statement is relevant to at least some of query."
    assert score <= 0.5, f"Score of {score} > 0.5. Statement is relevant to only some of query."

In [4]:
def test_lowadherence_medium():
    score = relevance("Name some famous dental floss brands","""Oral-B is an American brand of oral hygiene products, 
        including toothpastes, toothbrushes, electric toothbrushes, and mouthwashes. 
        The brand has been in business since the invention of the Hutson toothbrush in 1950 and in Redwood City, California.""")
    assert score >= 0.2, f"Score of {score} < 0.2. Statement is relevant to at least some of query."
    assert score <= 0.5, f"Score of {score} > 0.5. Statement is relevant to only some of query."

In [5]:
def test_lowadherence_long():
    score = relevance("Name some famous dental floss brands","""Types of floss and alternative options. Dental floss is regarded as the gold standard — it’s been around the longest compared to other plaque-removing products, Hewlett said. 
    Moursi also added that most flossing research studies have been conducted with dental floss, so there’s a lot of data showing its effectiveness. But floss is not one-size-fits-all, 
    he noted. Since using dental floss is difficult for some, there are other effective tools like interdental cleaners. Below, we broke down 
    the differences among several different options. Dental floss When people think of dental floss, it’s usually the threaded variety that comes on a spool. 
    But there’s also dental tape, which Hewlett described as a wider and flatter type of floss. He said it's particularly useful for 
    people with larger spaces between their teeth since it covers more surface area. Both forms of floss come in unflavored or flavored varieties, 
    but choosing a flavored option has no impact on how well it cleans your teeth, Hewlett said. Flosses also come waxed and unwaxed — 
    while a wax coating can make floss pass between teeth more easily, Hewitt said, both waxed and unwaxed are equally effective when used properly. 
    Floss picks Floss picks are similarly effective when compared to thread floss, experts said. The picks look like a wand and have a small piece 
    of floss at the forked end, so you can grip the handle while using the tool. Experts said floss picks are generally easy to use, especially if
    you’re flossing a child’s teeth. Water flossers Water flossers are powered devices that shoot pressurized water at the spaces between teeth, 
    targeting debris to disrupt and flush out plaque. While there is evidence to support their ability to remove plaque from teeth, Moursi
    said for water flossers to do their job, “you have to hold it in just the right place, at just the right angle and for just the right
    amount of time,” which can be challenging. Anyone can use water flossers, but experts said they’re the most beneficial for people who
    have difficulty using thread floss or floss threaders, as well as those with certain dental work like braces, bridges and crowns. 
    Interdental brushes Dental work like braces, bridges and crowns can block floss from slipping between teeth, making flossing challenging.
    Interdental brushes — which look like little spoolie brushes — can pass through the spaces between teeth and under any dental work,
    allowing you to remove plaque. The brushes have bristles on one end and a handle to grip on the other. To use, you point the brush at
    the gum line between teeth and push it through, moving the bristles around the space to remove plaque, said Hewlett. 
    The brushes come in various shapes and sizes to fit the spaces between your teeth.""")
    assert score >= 0.2, f"Score of {score} < 0.2. Statement is relevant to at least some of query."
    assert score <= 0.5, f"Score of {score} > 0.5. Statement is relevant to only some of query."

In [6]:
def test_majorityadherence_short():
    score = relevance("Name some famous dental floss brands?","Some key companies operating in the dental floss market include Colgate and Water Pik.")
    assert score >= 0.5, f"Score of {score} < 0.5. Statement is relevant to most of query."
    assert score <= 0.8, f"Score of {score} > 0.8. Statement is not relevant to all of query."

In [7]:
def test_majorityadherence_medium():
    score = relevance("How does the social structure of a lion pride impact the genetic diversity and long-term survival of the species?","""A typical pride of lions consists of about six related females, their dependent offspring, and a “coalition” 
    of 2–3 resident males that joined the pride from elsewhere. The pride is a “fission-fusion” society and
    pridemates are seldom found together, except for mothers that have pooled their offspring into a “crèche.”""")
    assert score >= 0.5, f"Score of {score} < 0.5. Statement is relevant to most of query."
    assert score <= 0.8, f"Score of {score} > 0.8. Statement is not relevant to all of query."

In [8]:
def test_majorityadherence_long():
    score = relevance("What are the parts of a cow's digestive tract, and how do they work including mouth, esophagus, the stomach, and intestines.","""
The cow's digestive tract consists of the following.

Mouth
A four-compartment stomach, which includes
The rumen (paunch)
The reticulum (“honeycomb”)
The omasum (“manyplies”)
The abomasum (“true stomach”)
Small intestine
Large intestine

The rumen
The rumen (on the left side of the animal) is the largest stomach compartment and consists of several sacs. It can hold 25 gallons or more of material depending on the size of the cow. Because of its size, the rumen acts as a storage or holding vat for feed.

Aside from storage, the rumen is also a fermentation vat. The rumen’s environment favors the growth of microbes. These microbes digest or ferment feed within the rumen and make volatile fatty acids (VFAs). The rumen absorbs most of the VFAs from fermentation.

A good blood supply to the rumen walls improves absorption of VFAs and other digestion products. Tiny projections (papillae) line the rumen, which increases the rumen’s surface area and the amount it can absorb.    

The reticulum 
The reticulum is a pouch-like structure in the forward area of the body, close to the heart. The tissues in the reticulum form a network similar to a honeycomb. A small tissue fold lies between the reticulum and rumen, but the two aren’t separate compartments. Together they’re called the rumino-reticulum.

Heavy or dense feed and metal objects eaten by the cow drop into this compartment. Nails and other sharp objects may work into the tissue and cause “hardware disease.” You can use magnets to prevent disease or correct the problem through surgery. Leaving it untreated may lead to infection and possibly death.

The omasum
The omasum is a globe-shaped structure containing leaves of tissue (like pages in a book). It absorbs water and other substances from digestive contents. Feed material (ingesta) between the leaves will be drier than ingesta found in the other compartments.

The abomasum
The abomasum is the only compartment lined with glands. These glands release hydrochloric acid and digestive enzymes, needed to breakdown feeds. The abomasum is similar to a nonruminant stomach.

The small intestine consists of three sections: the duodenum, jejunum and ileum. It measures about 20 times the length of the animal.

Secretions from the pancreas and gallbladder aid in digestion within the small intestine. The small intestine completes most of the digestive process and absorbs many nutrients through villi (small finger-like projections). From the villi the nutrients enter into the blood and lymphatic systems.

The cecum is the large area where the small and large intestine meet. The cecum breaks down some previously undigested fiber, but the exact importance of the cecum remains unknown.

The large intestine is the last section of the tract that undigested feedstuffs pass through. Microbes digest some undigested feed here, but the main digestive function of the large intestine is to absorb water.
""")
    assert score >= 0.5, f"Score of {score} < 0.5. Statement is relevant to most of query."
    assert score <= 0.8, f"Score of {score} > 0.8. Statement is not relevant to all of query."

### Long context with small relevant chunks are relevant

In [9]:
def test_smallrelevance_long():
    score = relevance("How rich was the authors boss?","""
all too keenly aware of the near-death experiences we seemed to have every few months. Nor had I changed my grad student lifestyle significantly since we started. So when Yahoo bought us it felt like going from rags to riches. Since we were going to California, I bought a car, a yellow 1998 VW GTI. I remember thinking that its leather seats alone were by far the most luxurious thing I owned.

The next year, from the summer of 1998 to the summer of 1999, must have been the least productive of my life. I didn't realize it at the time, but I was worn out from the effort and stress of running Viaweb. For a while after I got to California I tried to continue my usual m.o. of programming till 3 in the morning, but fatigue combined with Yahoo's prematurely aged culture and grim cube farm in Santa Clara gradually dragged me down. After a few months it felt disconcertingly like working at Interleaf.

Yahoo had given us a lot of options when they bought us. At the time I thought Yahoo was so overvalued that they'd never be worth anything, but to my astonishment the stock went up 5x in the next year. I hung on till the first chunk of options vested, then in the summer of 1999 I left. It had been so long since I'd painted anything that I'd half forgotten why I was doing this. My brain had been entirely full of software and men's shirts for 4 years. But I had done this to get rich so I could paint, I reminded myself, and now I was rich, so I should go paint.

When I said I was leaving, my boss at Yahoo had a long conversation with me about my plans. I told him all about the kinds of pictures I wanted to paint. At the time I was touched that he took such an interest in me. Now I realize it was because he thought I was lying. My options at that point were worth about $2 million a month. If I was leaving that kind of money on the table, it could only be to go and start some new startup, and if I did, I might take people with me. This was the height of the Internet Bubble, and Yahoo was ground zero of it. My boss was at that moment a billionaire. Leaving then to start a new startup must have seemed to him an insanely, and yet also plausibly, ambitious plan.

But I really was quitting to paint, and I started immediately. There was no time to lose. I'd already burned 4 years getting rich. Now when I talk to founders who are leaving after selling their companies, my advice is always the same: take a vacation. That's what I should have done, just gone off somewhere and done nothing for a month or two, but the idea never occurred to me.

So I tried to paint, but I just didn't seem to have any energy or ambition. Part of the problem was that I didn't know many people in California. I'd compounded this problem by buying a house up in the Santa Cruz Mountains, with a beautiful view but miles from anywhere. I stuck it out for a few more months, then in desperation I went back to New York, where unless you understand about rent control you'll be surprised to hear I still had my apartment, sealed up like a tomb of my old life. Idelle was in New York at least, and there were other people trying to paint there, even though I didn't know any of them.

When I got back to New York I resumed my old life, except now I was rich. It was as weird as it sounds. I resumed all my old patterns, except now there were doors where there hadn't been. Now when I was tired of walking, all I had to do was raise my hand, and (unless it was raining) a taxi would stop to pick me up. Now when I walked past charming little restaurants I could go in and order lunch. It was exciting for a while. Painting started to go better. I experimented with a new kind of still life where I'd paint one painting in the old way, then photograph it and print it, blown up, on canvas, and then use that as the underpainting for a second still life, painted from the same objects (which hopefully hadn't rotted
""")
    assert score >= 0.5, f"Score of {score} < 0.5. Relevant context is contained in long text."

## Context that provides no answer can still be relevant.

In [10]:
def test_nonanswer_short():
    score = relevance("How many countries are there in the world?", "There is no universally accepted answer to how many countries there are in the world.")
    assert score >= 0.5, f"Score of {score} < 0.5. Relevant context without definitive answer did not get a score of >= 0.5"

In [11]:
def test_nonanswer_medium():
    score = relevance("What is the meaning of life?", """No one can tell the actual definition of the meaning of life.
    For some, it is all about happiness, building a family, and leading life as it is. For some, it is about accumulating wealth, whereas,
    for some, it is all about love.""")
    assert score >= 0.5, f"Score of {score} < 0.5. Relevant context without definitive answer did not get a score of >= 0.5"

In [12]:
def test_nonanswer_long():
    score = relevance("What came first, the chicken or the egg?","""Eggs come from chickens and chickens come from eggs: that’s the basis of this ancient riddle. But eggs – which are just female sex cells – evolved more than a billion years ago, whereas chickens have been around for just 10,000 years. So the riddle is easily solved…or is it?

Taken at face value, there is no doubt that the egg came before the chicken. We tend to think of eggs as the shelled orbs laid by birds from which their chicks hatch – unless we eat them first. But all sexually reproducing species make eggs (the specialised female sex cells). That’s 99.99 per cent of all eukaryotic life – meaning organisms that have cells with a nucleus, so all animals and plants, and everything but the simplest life forms.

We don’t know for sure when sex evolved but it could have been as much as 2 billion years ago, and certainly more than 1 billion. Even the specialised sort of eggs laid by birds, with their tough outer membrane, evolved more than 300 million years ago.

As for chickens, they came into being much later. They are domesticated animals, so evolved as the result of humans purposefully selecting the least aggressive wild birds and letting them breed. This seems to have happened in several places independently, starting around 10,000 years ago.

The wild ancestor of chickens is generally agreed to be a tropical bird still living in the forests of Southeast Asia called the red junglefowl –  with other junglefowl species possibly adding to the genetic mix. From these origins, humans have carried chickens around the world over the past two millennia or more.

So, eggs dramatically predate chickens. But to be fair to the spirit of the riddle, we should also consider whether a chicken’s egg predates a chicken. As humans consistently chose the tamest red junglefowls and bred them together, the genetic makeup of the resulting birds will have shifted. At some stage during this domestication process the red junglefowl (Gallus gallus) evolved into a new subspecies, Gallus gallus domesticus, AKA the chicken.

In practice, it is impossible to pinpoint the moment when this happened. But in theory, at some point two junglefowl bred and their offspring was genetically different enough from the species of its parents to be classified as a chicken. This chicken would have developed within a junglefowl egg and only produced the very first chicken’s egg on reaching maturity. Looked at this way, the chicken came first.

"""
)
    assert score >= 0.5, f"Score of {score} < 0.5. Relevant context without definitive answer did not get a score of >= 0.5"

## Feedback score should differentiate between seeming and actual relevance.

In [13]:
def test_seemingrelevance_short():
    seemingly_relevant_score = relevance("Who won the superbowl in 2009?", "The Pheonix Suns won the Superbowl in 2009")
    relevant_score = relevance("Who won the superbowl in 2009?", "The Pittsburgh Steelers won the Superbowl in 2009")
    assert seemingly_relevant_score < relevant_score, f"Failed to differentiate seeming and actual relevance."

In [14]:
def test_seemingrelevance_medium():
    seemingly_relevant_score = relevance("What is a cephalopod?", """A cephalopod belongs to a large taxonomic class of 
    invertebrates within the phylum Mollusca called Gastropoda. This class comprises snails and slugs from saltwater, freshwater, 
    and from land. There are many thousands of species of sea snails and slugs, as well as freshwater snails, freshwater limpets, 
    and land snails and slugs.""")
    relevant_score = relevance("What is a cephalopod?", "A cephalopod is any member of the molluscan class Cephalopoda such as a squid, octopus, cuttlefish, or nautilus. These exclusively marine animals are characterized by bilateral body symmetry, a prominent head, and a set of arms or tentacles (muscular hydrostats) modified from the primitive molluscan foot. Fishers sometimes call cephalopods 'inkfish referring to their common ability to squirt ink.")
    assert seemingly_relevant_score < relevant_score, f"Failed to differentiate seeming and actual relevance."

In [15]:
def test_seemingrelevance_long():
    seemingly_relevant_score = relevance("Describe Abraham Lincoln's childhood", """
    Abraham Lincoln, Jr. was born on May 29, 1917, in Brookline, Massachusetts. His parents were Joseph P. Kennedy, Sr. and Rose Fitzgerald Kennedy. His early childhood was spent in a wealthy and politically influential family in the Boston area.

Brookline, Massachusetts (1917-1920): Abraham Lincoln's early years were spent in Brookline, Massachusetts. His father, Joseph P. Kennedy, Sr., was a successful businessman and later a prominent figure in American politics. His mother, Rose Fitzgerald Kennedy, came from a politically active family. Abraham Lincoln was the second of nine children.

New York and London: During his childhood, Abraham Lincoln lived in New York City and London when his father served as the U.S. ambassador to the United Kingdom. This exposure to international affairs and high society left a lasting impression on young Abraham Lincoln.

Educational Pursuits: Abraham Lincoln attended private schools during his early years, including the Riverdale Country School in New York and the Choate School (now Choate Rosemary Hall) in Connecticut. Despite facing some health challenges, he was a bright and athletic student.

Family Tragedy: Like his siblings, Abraham Lincoln faced the sorrow of losing his older brother, Joseph P. Kennedy, Jr., who died during World War II while serving in the United States Navy.

Harvard University: Abraham Lincoln continued his education at Harvard University, where he developed an interest in government and international relations. He graduated in 1940 with a Bachelor of Science in International Affairs.

Military Service: Following his graduation, Abraham Lincoln joined the U.S. Navy and served during World War II. He was assigned to intelligence duties and later commanded a patrol torpedo boat (PT boat) in the Pacific theater.

Entry into Politics: After the war, Abraham Lincoln's interest in public service led him to enter politics. He successfully ran for the U.S. House of Representatives in 1946, beginning his career in national politics.

Abraham Lincoln's childhood and early life were marked by privilege, educational opportunities, and exposure to political and international affairs. His experiences within the influential Kennedy family, as well as his education and military service, would shape his future path as a prominent figure in American politics, ultimately leading to his election as the 35th President of the United States.""")
    relevant_score = relevance("Describe Abraham Lincoln's childhood", """
    Abraham Lincoln was born on February 12, 1809, in a one-room log cabin on the Sinking Spring Farm in Hardin County, Kentucky (now part of LaRue County). His parents were Thomas Lincoln and Nancy Hanks Lincoln. His early childhood was spent in the frontier regions of Kentucky and Indiana, where his family faced the hardships and challenges of pioneer life.

Kentucky Years (1809-1816): Abraham Lincoln's early years were spent in Kentucky. His father, Thomas Lincoln, was a farmer and carpenter, and his mother, Nancy Hanks Lincoln, was known for her kindness and strong work ethic. Sadly, Nancy died when Abraham was just nine years old. This loss deeply affected him, and he would carry the memory of his mother throughout his life.

Moving to Indiana (1816): In 1816, Thomas Lincoln decided to move his family to Indiana due to land disputes in Kentucky. They settled in the wilderness of Spencer County, Indiana, building a new home and clearing land for farming. Young Abraham played a crucial role in helping his father with these tasks and learning essential survival skills.

Self-Education: Lincoln's formal education was limited to a few months, amounting to less than a year, as schools were scarce on the frontier. However, he was an avid reader and had a strong thirst for knowledge. Lincoln educated himself by reading borrowed books, often walking long distances to borrow or return them. He later referred to this time as having been "raised to farm work."

Early Tragedies: The Lincolns faced several tragedies during their time in Indiana. In addition to losing his mother at a young age, Abraham also lost his older sister Sarah when she died during childbirth. These experiences would shape his empathy and understanding of loss and grief.

Beginning His Legal Career: In 1830, the Lincoln family moved to Illinois, where Abraham Lincoln began to work as a farm laborer, rail-splitter, and store clerk. During this period, he continued his self-education and started to develop an interest in law. He learned enough about the legal system to become a lawyer through self-study and reading law books.

Marriage and Family: In 1842, Lincoln married Mary Todd, and they had four sons: Robert, Edward, William, and Thomas. Tragically, only one of their sons, Robert, survived to adulthood.

Abraham Lincoln's childhood and early life were marked by poverty, hard work, and the challenges of frontier living. Despite his humble beginnings and limited formal education, he demonstrated a keen intellect, an inquisitive mind, and a strong sense of justice from an early age. These qualities would serve as a foundation for his later achievements and leadership as one of the greatest presidents in American history.
    """)
    pass

## Relevant but inconclusive statements should get increasingly high scores as they are more helpful for answering the query.

In [16]:
def test_increasingrelevance_short():
    score_low = relevance("Who won the superbowl in 2009?","Santonio Holmes made a brilliant catch for the Steelers.")
    score_medium = relevance("Who won the superbowl in 2009?","Santonio Holmes made a brilliant catch for the Steelers in the superbowl.")
    score_high = relevance("Who won the superbowl in 2009?","Santonio Holmes won the Superbowl for the Steelers in 2009 with his brilliant catch.")
    assert (score_low < score_medium) & (score_medium < score_high), "Score did not increase with more relevant details."

In [17]:
def test_increasingrelevance_medium():
    score_low = relevance("What is a cephalopod?","Squids are a member of the molluscan class")
    score_medium = relevance("What is a cephalopod?","Squids are a member of the molluscan class characterized by bilateral body symmetry, a prominent head, and a set of arms or tentacles (muscular hydrostats) modified from the primitive molluscan foot.")
    score_high = relevance("What is a cephalopod?","A cephalopod is any member of the molluscan class such as squid, octopus or cuttlefish. These exclusively marine animals are characterized by bilateral body symmetry, a prominent head, and a set of arms or tentacles (muscular hydrostats) modified from the primitive molluscan foot.")
    assert (score_low < score_medium) & (score_medium < score_high), "Score did not increase with more relevant details."

In [18]:
def test_increasingrelevance_long():
    score_low = relevance("Describe Abraham Lincoln's childhood","Lincoln spent many of his early years in kentucky")
    score_medium = relevance("Describe Abraham Lincoln's childhood","""
    Lincoln was born in a one-room log cabin in Hardin County, Kentucky. His parents were Thomas Lincoln and Nancy Hanks Lincoln. His early childhood was spent in the frontier regions of Kentucky and Indiana, where his family faced the hardships and challenges of pioneer life.
    Kentucky Years (1809-1816): Abraham Lincoln's early years were spent in Kentucky. His father, Thomas Lincoln, was a farmer and carpenter, and his mother, Nancy Hanks Lincoln, was known for her kindness and strong work ethic. Sadly, Nancy died when Abraham was just nine years old. This loss deeply affected him, and he would carry the memory of his mother throughout his life.
    Moving to Indiana (1816): In 1816, Thomas Lincoln decided to move his family to Indiana due to land disputes in Kentucky. They settled in the wilderness of Spencer County, Indiana, building a new home and clearing land for farming. Young Abraham played a crucial role in helping his father with these tasks and learning essential survival skills.
    """)
    score_high = relevance("Describe Abraham Lincoln's childhood", """
    Abraham Lincoln was born on February 12, 1809, in a one-room log cabin on the Sinking Spring Farm in Hardin County, Kentucky (now part of LaRue County). His parents were Thomas Lincoln and Nancy Hanks Lincoln. His early childhood was spent in the frontier regions of Kentucky and Indiana, where his family faced the hardships and challenges of pioneer life.

Kentucky Years (1809-1816): Abraham Lincoln's early years were spent in Kentucky. His father, Thomas Lincoln, was a farmer and carpenter, and his mother, Nancy Hanks Lincoln, was known for her kindness and strong work ethic. Sadly, Nancy died when Abraham was just nine years old. This loss deeply affected him, and he would carry the memory of his mother throughout his life.

Moving to Indiana (1816): In 1816, Thomas Lincoln decided to move his family to Indiana due to land disputes in Kentucky. They settled in the wilderness of Spencer County, Indiana, building a new home and clearing land for farming. Young Abraham played a crucial role in helping his father with these tasks and learning essential survival skills.

Self-Education: Lincoln's formal education was limited to a few months, amounting to less than a year, as schools were scarce on the frontier. However, he was an avid reader and had a strong thirst for knowledge. Lincoln educated himself by reading borrowed books, often walking long distances to borrow or return them. He later referred to this time as having been "raised to farm work."

Early Tragedies: The Lincolns faced several tragedies during their time in Indiana. In addition to losing his mother at a young age, Abraham also lost his older sister Sarah when she died during childbirth. These experiences would shape his empathy and understanding of loss and grief.

Beginning His Legal Career: In 1830, the Lincoln family moved to Illinois, where Abraham Lincoln began to work as a farm laborer, rail-splitter, and store clerk. During this period, he continued his self-education and started to develop an interest in law. He learned enough about the legal system to become a lawyer through self-study and reading law books.

Marriage and Family: In 1842, Lincoln married Mary Todd, and they had four sons: Robert, Edward, William, and Thomas. Tragically, only one of their sons, Robert, survived to adulthood.

Abraham Lincoln's childhood and early life were marked by poverty, hard work, and the challenges of frontier living. Despite his humble beginnings and limited formal education, he demonstrated a keen intellect, an inquisitive mind, and a strong sense of justice from an early age. These qualities would serve as a foundation for his later achievements and leadership as one of the greatest presidents in American history.""")
    pass

## Run smoke tests

In [19]:
#! pip install tabulate

In [20]:
from tabulate import tabulate

def run_all_smoke_tests():
    smoke_tests = [
        test_lowadherence_short, test_lowadherence_medium, test_lowadherence_long,
        test_majorityadherence_short, test_majorityadherence_medium, test_majorityadherence_long, test_smallrelevance_long,
        test_nonanswer_short, test_nonanswer_medium, test_nonanswer_long,
        test_increasingrelevance_short, test_increasingrelevance_medium, test_increasingrelevance_long,
        test_seemingrelevance_short, test_seemingrelevance_medium, test_seemingrelevance_long
    ]

    total_tests = len(smoke_tests)
    total_passed = 0
    total_failed = 0
    type_passed = {}
    type_failed = {}
    length_passed = {}
    length_failed = {}

    for i, test in enumerate(smoke_tests):
        try:
            test()
            total_passed += 1

            # Update type_passed and length_passed dictionaries
            test_name = test.__name__
            test_type, test_length = test_name.split('_')[1], test_name.split('_')[2]
            type_passed[test_type] = type_passed.get(test_type, 0) + 1
            length_passed[test_length] = length_passed.get(test_length, 0) + 1
        except AssertionError as e:
            total_failed += 1

            # Update type_failed and length_failed dictionaries
            test_name = test.__name__
            test_type, test_length = test_name.split('_')[1], test_name.split('_')[2]
            type_failed[test_type] = type_failed.get(test_type, 0) + 1
            length_failed[test_length] = length_failed.get(test_length, 0) + 1

    # Prepare data for the table
    overall_data = [["Total Tests", total_tests],
                    ["Total Passed ✅", total_passed],
                    ["Total Failed ❌", total_failed]]

    type_data = [[f"{test_type.capitalize()} Tests", type_passed.get(test_type, 0), type_failed.get(test_type, 0),
                  f"{(type_passed.get(test_type, 0) / (type_passed.get(test_type, 0) + type_failed.get(test_type, 0))) * 100:.2f}%"]
                 for test_type in sorted(set(type_passed.keys()) | set(type_failed.keys()))]

    length_data = [[f"{test_length.capitalize()} Tests", length_passed.get(test_length, 0), length_failed.get(test_length, 0),
                    f"{(length_passed.get(test_length, 0) / (length_passed.get(test_length, 0) + length_failed.get(test_length, 0))) * 100:.2f}%"]
                   for test_length in sorted(set(length_passed.keys()) | set(length_failed.keys()))]

    return overall_data, type_data, length_data

# Get the data from the function
overall_data, type_data, length_data = run_all_smoke_tests()

In [21]:
# Print the table
print("\nOverall Results:")
print(tabulate(overall_data, headers=["Metric", "Count"], tablefmt="grid"))


Overall Results:
+-----------------+---------+
| Metric          |   Count |
+=================+=========+
| Total Tests     |      16 |
+-----------------+---------+
| Total Passed ✅ |      14 |
+-----------------+---------+
| Total Failed ❌ |       2 |
+-----------------+---------+


In [22]:
print("\nResults by Type:")
print(tabulate(type_data, headers=["Test Type", "Passed ✅", "Failed ❌", "Percentage Passed"], tablefmt="grid"))


Results by Type:
+---------------------------+-------------+-------------+---------------------+
| Test Type                 |   Passed ✅ |   Failed ❌ | Percentage Passed   |
+===========================+=============+=============+=====================+
| Increasingrelevance Tests |           3 |           0 | 100.00%             |
+---------------------------+-------------+-------------+---------------------+
| Lowadherence Tests        |           3 |           0 | 100.00%             |
+---------------------------+-------------+-------------+---------------------+
| Majorityadherence Tests   |           2 |           1 | 66.67%              |
+---------------------------+-------------+-------------+---------------------+
| Nonanswer Tests           |           3 |           0 | 100.00%             |
+---------------------------+-------------+-------------+---------------------+
| Seemingrelevance Tests    |           3 |           0 | 100.00%             |
+-----------------------

In [23]:
print("\nResults by Length:")
print(tabulate(length_data, headers=["Test Length", "Passed ✅", "Failed ❌", "Percentage Passed"], tablefmt="grid"))


Results by Length:
+---------------+-------------+-------------+---------------------+
| Test Length   |   Passed ✅ |   Failed ❌ | Percentage Passed   |
+===============+=============+=============+=====================+
| Long Tests    |           4 |           2 | 66.67%              |
+---------------+-------------+-------------+---------------------+
| Medium Tests  |           5 |           0 | 100.00%             |
+---------------+-------------+-------------+---------------------+
| Short Tests   |           5 |           0 | 100.00%             |
+---------------+-------------+-------------+---------------------+
